# We used Discrete Wavelet Transformation (DWT) to complete this project
# Task 1, Test 1 for Data Summarization; 
# Task 2, Test 2 for Similarity Search

# Task 1: Time Series Data Summarization

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pywt # python library for wavelet transformation

## Time Serie Compression and Reconstruction

### Compress of a time_serie x by using Daubechies Wavelet

In [2]:
# compression of data set 'x' to half size of 'x', e.g. 256 -> 128
def compress(x):
    compressed_x, cD = pywt.dwt(x, 'db1')
    return compressed_x

### Reconstruction of a time_serie from y by using Daubechies Wavelet

In [3]:
# reconstruction of data set 'y' to twice size of 'y', e.g. 128 -> 256
def reconstruct(y):
    reconstructed_y = pywt.idwt(y, None, 'db1')
    return reconstructed_y

### Iterative Compression of a time_serie x

In [4]:
# Compress 'x' i times and get a [x.size / 2**i] size data
# e.g. using com_iterative(x, 3) on x which contains 256 float numbers,
#      we'll have a compressed [256 / 2**3] = 32 float numbers
def com_iterative(x, i):
    for k in range(i):
        compressed_x = compress(x)
        x = compressed_x
    return x

### Iterative Decompression from a time_serie y

In [5]:
# reconstruction of data set 'y' to [y.original_size * 2**i]
def rec_iterative(y, i):
    tmp = y
    for k in range(i):
        reconstructed_y = reconstruct(tmp)
        tmp = reconstructed_y
    return reconstructed_y

### Interfaces to facilitate using compression/reconstruction functions above

In [6]:
# Interface functions to encode/decode easily
def compress_query(x, size):
    if size == 16 or size == 32 or size == 64 or size == 128:
        return com_iterative(x, int(math.log(x.shape[0]/size, 2)))
    else:
        print("compress_query(x,size): size is not valid.")
        import os
        os.system('pause')

def decoder256(y):
    return rec_iterative(y, int(math.log(256/y.shape[0], 2)))

# Test 1: Compression and Reconstruction with given 50k time_series

## Load 50k time_series (seismic and synthetic)

In [7]:
import struct
# load data set: 50000 seismic signals
filename_seismic = 'seismic_size50k_len256_znorm.bin'
with open(filename_seismic, 'rb') as in_file:
    time_series_seismic = np.array(struct.unpack('f' * 50000 * 256, in_file.read())).reshape(-1,256)

# load data set: 50000 synthetic signals
filename_synthetic = 'synthetic_size50k_len256_znorm.bin'
with open(filename_synthetic, 'rb') as in_file:
    time_series_synthetic = np.array(struct.unpack('f' * 50000 * 256, in_file.read())).reshape(-1,256)

## Data compression and reconstruction for 50k seismic time_series

In [8]:
# Following code is to compress and reconstruct seismic data
# every time serie will be compressed from 256 to 128/64/32/16 
# then the compressed data will be "enlarged" from 128/64/32/16 to 256

# The variables below is used to SAVE time serie compression and reconstruction for seismic data:
'''
resized128_seismic = np.empty((0,128),float)
resized64_seismic = np.empty((0,64),float)
resized32_seismic = np.empty((0,32),float)
resized16_seismic = np.empty((0,16),float)

reconstructed_128to256_seismic = np.empty((0,256),float)
reconstructed_64to256_seismic = np.empty((0,256),float)
reconstructed_32to256_seismic = np.empty((0,256),float)
reconstructed_16to256_seismic = np.empty((0,256),float)
'''

# Initialize variables to record reconstruction error for 50k time_series
error_128_seismic = 0.0
error_64_seismic = 0.0
error_32_seismic = 0.0
error_16_seismic = 0.0

data_size = 50000
for i in range(data_size):
    
    x = time_series_seismic[i,] # x contains 256 float numbers
    
    # 256 -> 128
    resized_128 = compress_query(x, 128)
    reconstructed_128 = decoder256(resized_128)
    error_128_seismic = error_128_seismic + float(np.linalg.norm(x - reconstructed_128))
    
    # 256 -> 64
    resized_64 = compress_query(x, 64)
    reconstructed_64 = decoder256(resized_64)
    error_64_seismic = error_64_seismic + float(np.linalg.norm(x - reconstructed_64))
    
    # 256 -> 32
    resized_32 = compress_query(x, 32)
    reconstructed_32 = decoder256(resized_32)
    error_32_seismic = error_32_seismic + float(np.linalg.norm(x - reconstructed_32))
    
    # 256 -> 16
    resized_16 = compress_query(x, 16)
    reconstructed_16 = decoder256(resized_16)
    error_16_seismic = error_16_seismic + float(np.linalg.norm(x - reconstructed_16))
    
    # Following code is to save compression and reconstruction to an array,
    # saving data takes much time than you could imagine
    # Please use the saved data in the folder "compressed_50k", 
    # an simple instruction to load data in the "ReadMe" file.
    '''
    resized128_seismic = np.vstack((resized128_seismic, resized_128))
    reconstructed_128to256_seismic = np.vstack((reconstructed_128to256_seismic, reconstructed_128))
    resized64_seismic = np.vstack((resized64_seismic, resized_64))
    reconstructed_64to256_seismic = np.vstack((reconstructed_64to256_seismic, reconstructed_64))
    resized32_seismic = np.vstack((resized32_seismic, resized_32))
    reconstructed_32to256_seismic = np.vstack((reconstructed_32to256_seismic,reconstructed_32))
    resized16_seismic = np.vstack((resized16_seismic, resized_16))
    reconstructed_16to256_seismic = np.vstack((reconstructed_16to256_seismic,reconstructed_16))

np.save("compressed128_50k_seismic.npy", resized128_seismic)
np.save("compressed64_50k_seismic.npy", resized64_seismic)
np.save("compressed32_50k_seismic.npy", resized32_seismic)
np.save("compressed16_50k_seismic.npy", resized16_seismic)

np.save("reconstructed_128to256_50k_seismic.npy", reconstructed_128to256_seismic)
np.save("reconstructed_64to256_50k_seismic.npy", reconstructed_64to256_seismic)
np.save("reconstructed_32to256_50k_seismic.npy", reconstructed_32to256_seismic)
np.save("reconstructed_16to256_50k_seismic.npy", reconstructed_16to256_seismic)
'''

# Print the reconstruction error for 50k time_series
print("Overall reconstruction error for ", data_size, " seismic time series: (128 / 64 / 32 / 16)")
print(error_128_seismic, error_64_seismic, error_32_seismic, error_16_seismic)
print("Average reconstruction error for ", data_size, " seismic time series: (128 / 64 / 32 / 16)")
print(error_128_seismic/data_size, error_64_seismic/data_size, error_32_seismic/data_size, error_16_seismic/data_size)

Overall reconstruction error for  50000  seismic time series: (128 / 64 / 32 / 16)
632932.170386034 771074.4015698319 790471.3970702328 797346.2607445099
Average reconstruction error for  50000  seismic time series: (128 / 64 / 32 / 16)
12.658643407720678 15.421488031396638 15.809427941404655 15.946925214890197


## Data compression and reconstruction for 50k synthetic time_series

In [9]:
# Following code is to compress and reconstruct seismic data
# every time serie will be compressed from 256 to 128/64/32/16 
# then the compressed data will be "enlarged" from 128/64/32/16 to 256


# The variables below is used to SAVE time serie compression and reconstruction for 50k synthetic time_series:
'''
resized128_synthetic = np.empty((0,128),float)
resized64_synthetic = np.empty((0,64),float)
resized32_synthetic = np.empty((0,32),float)
resized16_synthetic = np.empty((0,16),float)

reconstructed_128to256_synthetic = np.empty((0,256),float)
reconstructed_64to256_synthetic = np.empty((0,256),float)
reconstructed_32to256_synthetic = np.empty((0,256),float)
reconstructed_16to256_synthetic = np.empty((0,256),float)
'''

# Initialize variables to record reconstruction error for 50k time_series
error_128_synthetic = 0.0
error_64_synthetic = 0.0
error_32_synthetic = 0.0
error_16_synthetic = 0.0

data_size = 50000
for i in range(data_size):
    
    x = time_series_synthetic[i,] # x contains 256 float numbers
    
    # 256 -> 128
    resized_128 = compress_query(x, 128)
    reconstructed_128 = decoder256(resized_128)
    error_128_synthetic = error_128_synthetic + float(np.linalg.norm(x - reconstructed_128))
    
    # 256 -> 64
    resized_64 = compress_query(x, 64)
    reconstructed_64 = decoder256(resized_64)
    error_64_synthetic = error_64_synthetic + float(np.linalg.norm(x - reconstructed_64))
    
    # 256 -> 32
    resized_32 = compress_query(x, 32)
    reconstructed_32 = decoder256(resized_32)
    error_32_synthetic = error_32_synthetic + float(np.linalg.norm(x - reconstructed_32))
    
    # 256 -> 16
    resized_16 = compress_query(x, 16)
    reconstructed_16 = decoder256(resized_16)
    error_16_synthetic = error_16_synthetic + float(np.linalg.norm(x - reconstructed_16))
    
    # Following code can save compression and reconstruction to an array, saving data takes much time than you could imagine.
    # Please use the saved data in the folder "compressed_50k", an simple instruction to load data in the "ReadMe" file.
    '''
    resized128_synthetic = np.vstack((resized128_synthetic, resized_128))
    reconstructed_128to256_synthetic = np.vstack((reconstructed_128to256_synthetic, reconstructed_128))
    resized64_synthetic = np.vstack((resized64_synthetic, resized_64))
    reconstructed_64to256_synthetic = np.vstack((reconstructed_64to256_synthetic, reconstructed_64))
    resized32_synthetic = np.vstack((resized32_synthetic, resized_32))
    reconstructed_32to256_synthetic = np.vstack((reconstructed_32to256_synthetic,reconstructed_32))
    resized16_synthetic = np.vstack((resized16_synthetic, resized_16))
    reconstructed_16to256_synthetic = np.vstack((reconstructed_16to256_synthetic,reconstructed_16))

np.save("compressed128_50k_synthetic.npy", resized128_synthetic)
np.save("compressed64_50k_synthetic.npy", resized64_synthetic)
np.save("compressed32_50k_synthetic.npy", resized32_synthetic)
np.save("compressed16_50k_synthetic.npy", resized16_synthetic)

np.save("reconstructed_128to256_50k_synthetic.npy", reconstructed_128to256_synthetic)
np.save("reconstructed_64to256_50k_synthetic.npy", reconstructed_64to256_synthetic)
np.save("reconstructed_32to256_50k_synthetic.npy", reconstructed_32to256_synthetic)
np.save("reconstructed_16to256_50k_synthetic.npy", reconstructed_16to256_synthetic)
'''

# Print the reconstruction error for 50k time_series
print("Overall reconstruction error for ", data_size, " synthetic time series: (128 / 64 / 32 / 16)")
print(error_128_synthetic, error_64_synthetic, error_32_synthetic, error_16_synthetic)
print("Average reconstruction error for ", data_size, " synthetic time series: (128 / 64 / 32 / 16)")
print(error_128_synthetic/data_size, error_64_synthetic/data_size, error_32_synthetic/data_size, error_16_synthetic/data_size)

Overall reconstruction error for  50000  synthetic time series: (128 / 64 / 32 / 16)
76273.23288734269 120327.73912184678 173555.64466821024 244335.4583699993
Average reconstruction error for  50000  synthetic time series: (128 / 64 / 32 / 16)
1.5254646577468538 2.4065547824369355 3.4711128933642046 4.886709167399986


# Task 2: Similarity Search by using Euclidean Distance (with Data Summarization)

## Similarity Search by using Euclidean Distance

In [10]:
def similarity_euclidean(query, time_series, compress_size):
    
    # Compress the query signal to a smaller time serie which contains "compress_size" float numbers
    compressed_query = compress_query(query, compress_size)
    
    # Initialize compressed minimum distance, original minimum distance as infinitiy (here we set as 10000.0)
    D_closest_com = 10000.0
    D_closest_orginal = 10000.0
    # Initialize closest time serie's index as 0
    index_closest = 0
    # Initialize pruning count as 0
    pruning_count = 0
    
    # Loop for all time_series
    for i in range(time_series.shape[0]):
        
        # Compress i-th time serie as x
        x = compress_query(time_series[i,], compress_size)
        # Calculate euclidean distance between compressed time series
        dist_com = float(np.linalg.norm(x - compressed_query))
        
        # If the euclidean distance between compressed time series is smaller
        if D_closest_com > dist_com:
            
            # Calculate the euclidean distance between original time series
            dist_real = float(np.linalg.norm(query - time_series[i,]))
            
            # If the euclidean distance between original time series is smaller
            if D_closest_orginal > dist_real:
                
                # Memorization
                D_closest_com = dist_com
                D_closest_orginal = dist_real
                index_closest = i
                
        # When euclidean distance between compressed time series is bigger
        else:
            # The time serie is pruned, we count all pruned times
            pruning_count += 1
            
    # Return index, distance and pruning times for this similarity search
    return index_closest, D_closest_orginal, pruning_count

# Test 2: Similarity Search with 

## Load 100 query time_series (seismic and synthetic)

In [11]:
import struct
# load data set: 100 seismic signals
filename_seismic = 'seismic-query_size100_len256_znorm.bin'
with open(filename_seismic, 'rb') as in_file:
    time_series_seismic_query = np.array(struct.unpack('f' * 100 * 256, in_file.read())).reshape(-1,256)

# load data set: 100 synthetic signals
filename_synthetic = 'synthetic-query_size100_len256_znorm.bin'
with open(filename_synthetic, 'rb') as in_file:
    time_series_synthetic_query = np.array(struct.unpack('f' * 100 * 256, in_file.read())).reshape(-1,256)

## Load 50k time_series for search

In [12]:
import struct
# load data set: 50000 seismic signals
filename_seismic = 'seismic_size50k_len256_znorm.bin'
with open(filename_seismic, 'rb') as in_file:
    time_series_seismic = np.array(struct.unpack('f' * 50000 * 256, in_file.read())).reshape(-1,256)

# load data set: 50000 synthetic signals
filename_synthetic = 'synthetic_size50k_len256_znorm.bin'
with open(filename_synthetic, 'rb') as in_file:
    time_series_synthetic = np.array(struct.unpack('f' * 50000 * 256, in_file.read())).reshape(-1,256)

## Similarity Search of a time_serie by using the defined function "similarity_euclidean(query, time_series, compress_size)"

### Find a closest time_serie q with 50k seismic time_series, with compression ratio 16 (256/16)

In [13]:
ind_query = 3
q = time_series_seismic_query[ind_query,]

idx, dist, pruning_count = similarity_euclidean(q, time_series_seismic, 16)

print("The closest time_serie to the query is ", idx, "-th time_serie.") 
print("The closest distance is ", dist)
print("For this similarity search, ", pruning_count, " time_series have been pruned.")

The closest time_serie to the query is  12503 -th time_serie.
The closest distance is  0.0
For this similarity search,  49567  time_series have been pruned.
